## Using CUDA

In [1]:
import torch

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda
NVIDIA GeForce RTX 3070
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
# !isic image download --search "diagnosis:\"actinic keratosis\"" real_images/

## Organising Dataset

In [3]:
import os
import shutil

# Function copies images from one folder to another, given a startpoint and endpoint.
def copy_images(start_index, end_index, source_folder, destination_folder):
    image_files = os.listdir(source_folder)
    image_files = [file for file in image_files if file.lower().endswith('.jpg') or file.lower().endswith('.jpeg')]
    # image_files.sort()  # Sort the files alphabetically

    # Validate the range of indices
    if start_index < 0 or end_index >= len(image_files) or start_index > end_index:
        print("Invalid range of indices.")
        return

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Iterate over the image files in the given range
    for i in range(start_index, end_index + 1):
        image_file = image_files[i]
        source_path = os.path.join(source_folder, image_file)
        destination_path = os.path.join(destination_folder, image_file)

        # Move the image file to the destination folder
        shutil.copy(source_path, destination_path)

    print(f"Copied {end_index - start_index + 1} image(s) from '{source_folder}' to '{destination_folder}'.")

In [4]:
# Function moves necessary images to make training and testing folder structure
def prepare_data():
    # Available: 1000 real, 100 fake

    # Train data (85 real, 85 fake)
    copy_images(0, 84, "./real_images/", "./dataset/train/real")
    copy_images(0, 84, "./fake_images/", "./dataset/train/fake")

    # Test data (15 real, 15 fake)
    copy_images(85, 99, "./real_images/", "./dataset/test/real")
    copy_images(85, 99, "./fake_images/", "./dataset/test/fake")

    print(f"Copied images!")

prepare_data()

Copied 85 image(s) from './real_images/' to './dataset/train/real'.
Copied 85 image(s) from './fake_images/' to './dataset/train/fake'.
Copied 15 image(s) from './real_images/' to './dataset/test/real'.
Copied 15 image(s) from './fake_images/' to './dataset/test/fake'.
Copied images!


## Loading Dataset

In [5]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import os

class SkinCancerDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Collect image paths and corresponding labels
        for folder in os.listdir(root_dir):
            folder_path = os.path.join(root_dir, folder)
            if os.path.isdir(folder_path):
                label = 0 if folder == 'real' else 1
                for image_name in os.listdir(folder_path):
                    image_path = os.path.join(folder_path, image_name)
                    self.image_paths.append(image_path)
                    self.labels.append(label)
    
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path).convert('RGB')
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label


In [6]:
from torchvision.transforms import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Example usage
train_dataset = SkinCancerDataset('dataset/train', transform=transform)
test_dataset = SkinCancerDataset('dataset/test', transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

## Model
Pre-trained Res-Net 18 Model

In [7]:
import torchvision.models as models
import torch.nn as nn

# Load a pre-trained model
model = models.resnet18(pretrained=True)

# Replace the last fully-connected layer
# Parameters of newly constructed modules have requires_grad=True by default
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2) # binary classification (real/fake)

# Move model to GPU if available
model = model.to(device)

c:\Users\fdabr\anaconda3\envs\machine_learning\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\fdabr\anaconda3\envs\machine_learning\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

## Training

In [9]:
print("Using device:", device)

model.train()
num_epochs = 2
print_interval = 10  # Adjust value to control how often to print loss

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss for this batch
        running_loss += loss.item()
        if (i + 1) % print_interval == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/print_interval:.4f}")
            running_loss = 0.0
        
        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print accuracy for this epoch
    epoch_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {epoch_accuracy:.2f}%")


Using device: cuda
Epoch [1/2], Step [10/43], Loss: 0.9486
Epoch [1/2], Step [20/43], Loss: 1.2882
Epoch [1/2], Step [30/43], Loss: 0.4265
Epoch [1/2], Step [40/43], Loss: 0.3495
Epoch [1/2], Accuracy: 74.71%
Epoch [2/2], Step [10/43], Loss: 0.2158
Epoch [2/2], Step [20/43], Loss: 0.3950
Epoch [2/2], Step [30/43], Loss: 0.4459
Epoch [2/2], Step [40/43], Loss: 0.3301
Epoch [2/2], Accuracy: 87.06%


## Testing

In [10]:
# Switch the model to evaluation mode
model.eval()

with torch.no_grad():
    real_count = 0
    fake_count = 0
    correct = 0
    total = 0
    for images, labels in test_loader:
        real_count += len(labels[labels == 1])
        fake_count += len(labels[labels == 0])

        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f'Tested {real_count} real images, and {fake_count} fake images.')
    print(f'Test Accuracy of the model on the test images: {100 * correct / total} %')

Tested 15 real images, and 15 fake images.
Test Accuracy of the model on the test images: 86.66666666666667 %


In [11]:
from PIL import Image

# Function for testing single image
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')

    transform = transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image)
    image = image.unsqueeze(0).to(device)  # Add batch dimension and move to device

    model.eval()  # Switch to evaluation mode
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    if predicted.item() == 0:
        return "Real"
    else:
        return "Synthetic"


In [14]:
# Testing random images

real_list = os.listdir("./real_images/")
real_list = [file for file in real_list if file.lower().endswith('.jpg') or file.lower().endswith('.jpeg')]

fake_list = os.listdir("./fake_images/")
fake_list = [file for file in fake_list if file.lower().endswith('.jpg') or file.lower().endswith('.jpeg')]

print(predict_image("./real_images/" + real_list[948]))
print(predict_image("./fake_images/" + fake_list[0]))

Real
Synthetic
